In [1]:
!pip install youtube-transcript-api
!pip install llama-index
%pip install llama-index-embeddings-openai

    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
import dotenv
import os

In [16]:
# Load environment variables from file
dotenv.load_dotenv('./.env')
apiKey = os.getenv("OPENAI_API_KEY")

In [4]:

GPT_MODEL = 'gpt-4o'
embed_model = OpenAIEmbedding(model="text-embedding-3-small", embed_batch_size=2)
llm = OpenAI(model=GPT_MODEL, api_key=apiKey)

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
video_id = 'g3CvsPAF3_0'
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

In [6]:
print(transcript[0])

{'text': 'a good friend has recently shared that', 'start': 4.4, 'duration': 5.64}


In [7]:
# Combine all transcript segments into a single paragraph
combined_text = ' '.join([entry['text'] for entry in transcript])
print(combined_text[:102])

a good friend has recently shared that she's feeling tired like really tired not I did not get enough 


Now we output transcript into local folder.

In [8]:
# define function to output transcript into text file
def output_transcript(transcript, output_file):
    with open(output_file, 'w') as f:
        for entry in transcript:
            f.write(entry['text'] + '\n')

path_dir = './data-transcript/'

output_transcript(transcript, path_dir + video_id + '.txt')

In [9]:
from copy import deepcopy
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex


def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

In [10]:
# Load documents from the specified directory
documents = SimpleDirectoryReader(path_dir).load_data()

In [11]:
print(len(documents))

3


In [12]:
print(documents[1])

Doc ID: d99a0422-4fbd-4efc-a209-40a4e498dcb3
Text: a good friend has recently shared that she's feeling tired like
really tired not I did not get enough sleep tired but something deeper
than that she said she feels and I'm quoting as if I have lost my
motivation I call her bre because she's so clearly dedicated to a job
and conducts herself with professionalism and and kindness she's a
wonderful...


In [13]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

Sự nhẫn nhịn là khả năng kiên trì, chịu đựng và không phản ứng tiêu cực trước những khó khăn, thử thách hoặc sự không hài lòng. Nó bao gồm việc giữ bình tĩnh, không nổi nóng và duy trì thái độ tích cực trong mọi tình huống.


In [ ]:
response = query_engine.query("Sự nhẫn nhịn là gì ?")
print(response)

In [15]:
response = query_engine.query("Summarize main idea about uncomfortable feeling")
print(response)

The main idea about uncomfortable feelings is that embracing discomfort can lead to personal growth and increased daring. In a study with an improvisation class, participants who were encouraged to feel uncomfortable took the stage for longer and were more daring compared to those who were not. This suggests that seeking temporary discomfort can help individuals grow and improve their skills.
